# 1인가구 정보 확인을 위한 전처리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

drive_path = '/content/drive/MyDrive/new_project1/'

Mounted at /content/drive


In [ ]:
import pandas as pd
import re

import geopandas as gpd


# 제출용 코드에서 깔끔하게 보이게,..
import warnings
warnings.filterwarnings("ignore")

In [ ]:
raw = pd.read_excel(f'{drive_path}data/pop/2023.12월_10개 관심집단 수.xlsx')
raw.head()

,행정동코드,자치구,행정동명,성별,연령대,총인구,1인가구수,커뮤니케이션이 적은 집단,평일 외출이 적은 집단,휴일 외출이 적은 집단,출근소요시간 및 근무시간이 많은 집단,외출이 매우 적은 집단(전체),외출이 매우 많은 집단,동영상서비스 이용이 많은 집단,생활서비스 이용이 많은 집단,재정상태에 대한 관심집단,외출-커뮤니케이션이 모두 적은 집단(전체)
0,1101053,종로구,사직동,1,20,214.0,64.37,9.86,9.17,9.62,8.06,11.91,3.85,11.64,3.34,2.79,2.36
1,1101053,종로구,사직동,1,25,329.0,113.46,9.90,10.73,8.85,10.23,9.72,12.14,8.01,13.30,15.52,9.67
2,1101053,종로구,사직동,1,30,376.0,120.93,18.55,13.51,9.29,15.78,23.77,18.76,14.12,12.36,6.46,17.33
3,1101053,종로구,사직동,1,35,347.0,107.60,21.29,18.30,5.19,15.23,8.72,15.10,18.61,11.60,9.49,12.15
4,1101053,종로구,사직동,1,40,411.0,97.57,21.68,14.34,10.17,13.92,23.21,10.08,17.82,5.93,8.26,23.08


In [ ]:
pop = raw.copy()
pop = pop.iloc[:, :7]
pop.head(20)

,행정동코드,자치구,행정동명,성별,연령대,총인구,1인가구수
0,1101053,종로구,사직동,1,20,214.00,64.37
1,1101053,종로구,사직동,1,25,329.00,113.46
2,1101053,종로구,사직동,1,30,376.00,120.93
3,1101053,종로구,사직동,1,35,347.00,107.60
4,1101053,종로구,사직동,1,40,411.00,97.57
5,1101053,종로구,사직동,1,45,371.20,88.59
6,1101053,종로구,사직동,1,50,436.80,66.81
7,1101053,종로구,사직동,1,55,380.80,58.60
8,1101053,종로구,사직동,1,60,337.60,55.79
9,1101053,종로구,사직동,1,65,289.60,53.91


In [ ]:
## 1인 가구수, 총인구수 소수점 단위 반올림

pop['1인가구수'] = pop['1인가구수'].apply(lambda x: round(x))
pop['총인구'] = pop['총인구'].apply(lambda x: round(x))
pop.head()

,행정동코드,자치구,행정동명,성별,연령대,총인구,1인가구수
0,1101053,종로구,사직동,1,20,214,64
1,1101053,종로구,사직동,1,25,329,113
2,1101053,종로구,사직동,1,30,376,121
3,1101053,종로구,사직동,1,35,347,108
4,1101053,종로구,사직동,1,40,411,98


In [ ]:
df = pop.groupby(['행정동코드'])[['총인구', '1인가구수']].agg('sum')
df.columns = ['total_all', 'solo_all']
df.head()

,total_all,solo_all
행정동코드,,
1101053,8451,2081
1101054,2569,894
1101055,9202,1563
1101056,15577,1633
1101057,6418,810


In [ ]:
pop['연령대'].unique()

array([20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75])

In [ ]:
## 연령별 그룹화
age_groups = [x for x in range(20, 80, 10)]

for age in age_groups:
    temp = pop[pop['연령대'].isin([age, age + 5])]
    temp_group = temp.groupby(['행정동코드'])[['총인구', '1인가구수']].agg('sum')
    temp_group.columns = [f'total_{age}', f'solo_{age}']
    df = pd.concat([df, temp_group], axis = 1)

df.reset_index(inplace = True, drop = False)
df.head()

,행정동코드,total_all,solo_all,total_20,solo_20,total_30,solo_30,total_40,solo_40,total_50,solo_50,total_60,solo_60,total_70,solo_70
0,1101053,8451,2081,1273,435,1535,480,1659,389,1637,256,1236,219,1111,302
1,1101054,2569,894,486,242,393,175,415,129,498,129,387,91,390,128
2,1101055,9202,1563,1984,370,1330,320,1826,216,1758,167,1178,187,1126,303
3,1101056,15577,1633,2444,260,2194,247,3113,226,3400,233,2148,181,2278,486
4,1101057,6418,810,886,94,986,135,1549,101,1342,123,852,130,803,227


In [ ]:
solo = df.merge(raw[['행정동코드', '자치구', '행정동명']],
                how = 'left',
                on = '행정동코드')

solo['행정동명'] = solo['행정동명'].apply(lambda x: re.sub(r'\·', '', x))

solo.drop_duplicates(inplace = True)
solo.reset_index(drop = True, inplace = True)

solo.head()

,행정동코드,total_all,solo_all,total_20,solo_20,total_30,solo_30,total_40,solo_40,total_50,solo_50,total_60,solo_60,total_70,solo_70,자치구,행정동명
0,1101053,8451,2081,1273,435,1535,480,1659,389,1637,256,1236,219,1111,302,종로구,사직동
1,1101054,2569,894,486,242,393,175,415,129,498,129,387,91,390,128,종로구,삼청동
2,1101055,9202,1563,1984,370,1330,320,1826,216,1758,167,1178,187,1126,303,종로구,부암동
3,1101056,15577,1633,2444,260,2194,247,3113,226,3400,233,2148,181,2278,486,종로구,평창동
4,1101057,6418,810,886,94,986,135,1549,101,1342,123,852,130,803,227,종로구,무악동


In [ ]:
solo.isna().sum()

행정동코드        0
total_all    0
solo_all     0
total_20     0
solo_20      0
total_30     0
solo_30      0
total_40     0
solo_40      0
total_50     0
solo_50      0
total_60     0
solo_60      0
total_70     0
solo_70      0
자치구          0
행정동명         0
dtype: int64

In [ ]:
solo.head()

,행정동코드,total_all,solo_all,total_20,solo_20,total_30,solo_30,total_40,solo_40,total_50,solo_50,total_60,solo_60,total_70,solo_70,자치구,행정동명
0,1101053,8451,2081,1273,435,1535,480,1659,389,1637,256,1236,219,1111,302,종로구,사직동
1,1101054,2569,894,486,242,393,175,415,129,498,129,387,91,390,128,종로구,삼청동
2,1101055,9202,1563,1984,370,1330,320,1826,216,1758,167,1178,187,1126,303,종로구,부암동
3,1101056,15577,1633,2444,260,2194,247,3113,226,3400,233,2148,181,2278,486,종로구,평창동
4,1101057,6418,810,886,94,986,135,1549,101,1342,123,852,130,803,227,종로구,무악동


### 지도 파일과 결합

In [ ]:
def merge_map(df_name, key):

    region_code = pd.read_csv(f'{drive_path}final/data/region_code.csv')
    dong_shp = gpd.read_file(f'{drive_path}data/seoul_shp/dong/bnd_dong_11_2023_2023_2Q.shp')

    # 코드명 타입 일치
    region_code['ADM_CD'] = region_code['ADM_CD'].astype('int')
    dong_shp['ADM_CD'] = dong_shp['ADM_CD'].astype('int')

    # 지도파일과 지역코드 결합
    map_base = dong_shp.merge(region_code,
                          how = 'left',
                          on = 'ADM_CD')
    map_base['ADM_NM'] = map_base['ADM_NM'].apply(lambda x: re.sub(r'\·', '', x))

    # 동이름 다른것에 대응하기 위해서
    map_base['key1'] = map_base['시군구'] + ' ' + map_base['ADM_NM']
    map_base['key2'] = map_base['시군구'] + ' ' + map_base['읍면동']

    if '구동' in df_name.columns:
        pass
    else :
        df_name['구동'] = df_name['자치구'] + ' ' + df_name['행정동명']

    globals()['final'] = map_base.merge(df_name.rename(columns = {'구동' : key}),
                                        how = 'left',
                                        on = key)

    final.dropna(inplace = True)

In [ ]:
merge_map(solo, 'key1')

final.head()

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,...,total_40,solo_40,total_50,solo_50,total_60,solo_60,total_70,solo_70,자치구,행정동명
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",서울특별시 종로구 사직동,1101053.0,1.111053e+09,11010530.0,11110530.0,11110530.0,...,1659.0,389.0,1637.0,256.0,1236.0,219.0,1111.0,302.0,종로구,사직동
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1...",서울특별시 종로구 삼청동,1101054.0,1.111054e+09,11010540.0,11110540.0,11110540.0,...,415.0,129.0,498.0,129.0,387.0,91.0,390.0,128.0,종로구,삼청동
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1...",서울특별시 종로구 부암동,1101055.0,1.111055e+09,11010550.0,11110550.0,11110550.0,...,1826.0,216.0,1758.0,167.0,1178.0,187.0,1126.0,303.0,종로구,부암동
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1...",서울특별시 종로구 평창동,1101056.0,1.111056e+09,11010560.0,11110560.0,11110560.0,...,3113.0,226.0,3400.0,233.0,2148.0,181.0,2278.0,486.0,종로구,평창동
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1...",서울특별시 용산구 한남동,1103074.0,1.117068e+09,11030740.0,11170685.0,11170685.0,...,3422.0,919.0,3398.0,589.0,2666.0,521.0,2225.0,657.0,용산구,한남동


In [ ]:
final.columns

Index(['BASE_DATE', 'ADM_NM', 'ADM_CD', 'geometry', 'dong_name', '통계청코드',
       '도로명코드10', '통계청코드8', '도로명코드8', '도로명코드', '시도명', '시군구', '읍면동', 'key1',
       'key2', '행정동코드', 'total_all', 'solo_all', 'total_20', 'solo_20',
       'total_30', 'solo_30', 'total_40', 'solo_40', 'total_50', 'solo_50',
       'total_60', 'solo_60', 'total_70', 'solo_70', '자치구', '행정동명'],
      dtype='object')

In [ ]:
final[final.isna().any(axis=1)]

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,...,total_40,solo_40,total_50,solo_50,total_60,solo_60,total_70,solo_70,자치구,행정동명


In [ ]:
final.columns

Index(['BASE_DATE', 'ADM_NM', 'ADM_CD', 'geometry', 'dong_name', '통계청코드',
       '도로명코드10', '통계청코드8', '도로명코드8', '도로명코드', '시도명', '시군구', '읍면동', 'key1',
       'key2', '행정동코드', 'total_all', 'solo_all', 'total_20', 'solo_20',
       'total_30', 'solo_30', 'total_40', 'solo_40', 'total_50', 'solo_50',
       'total_60', 'solo_60', 'total_70', 'solo_70', '자치구', '행정동명'],
      dtype='object')

In [ ]:
selected_columns = ['ADM_CD'] + final.columns[15:].tolist()
selected_columns

['ADM_CD',
 '행정동코드',
 'total_all',
 'solo_all',
 'total_20',
 'solo_20',
 'total_30',
 'solo_30',
 'total_40',
 'solo_40',
 'total_50',
 'solo_50',
 'total_60',
 'solo_60',
 'total_70',
 'solo_70',
 '자치구',
 '행정동명']

In [ ]:
final = final[selected_columns]

In [ ]:
final.isna().sum()

ADM_CD       0
행정동코드        0
total_all    0
solo_all     0
total_20     0
solo_20      0
total_30     0
solo_30      0
total_40     0
solo_40      0
total_50     0
solo_50      0
total_60     0
solo_60      0
total_70     0
solo_70      0
자치구          0
행정동명         0
dtype: int64

In [ ]:
# csv로 저장
final.to_csv(f'{drive_path}final/data/singlehousehold.csv', index = False)